# [프로젝트] 멀티 에이전트 기반 보고서 작성과 품질 관리


이번 실습에서는, 기업의 재무 공시 정보와 최근 뉴스를 바탕으로 보고서를 작성하는 에이전트를 만들어 보겠습니다.   

이는 단순 검색보다는 최신성을 가진 뉴스나 공시 등의 전문 자료의 내용이 중요합니다.


In [ ]:
!pip install langgraph dotenv arxiv langchain-tavily langchain-community langchain-google-genai pymupdf -q


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install sentence-transformers chromadb langchain-chroma rank-bm25 pymupdf -q
# RAG


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


이번에는 공시 문서 검색을 위해 DART(https://opendart.fss.or.kr/) API 키가 필요합니다.  

해당 페이지에서 회원가입 후 API 키를 발급받습니다.   
해당 키는 .env의 DART_API_KEY 에 저장해 주세요.

In [ ]:
from dotenv import load_dotenv
import os

# GOOGLE_API_KEY, TAVILY_API_KEY, DART_API_KEY 필수
# LangSmith (선택)
load_dotenv()

True

## Preliminary

### DART 공시 문서 불러오기   
공시 문서를 불러올 수 있는 DART API와 같이, 실행할 때마다 파라미터가 달라지는 경우에는 툴로 구성하는 것이 효과적일 수 있습니다.  

DART 공시 문서를 API를 통해 불러오는 함수를 구성합니다.

In [ ]:
import os
import requests
from dotenv import load_dotenv
import zipfile
import io
from datetime import datetime, timedelta
import shutil
from langchain_core.tools import tool


def get_dart_documents(corp_name, start_date=None, period=180):
    """
    DART API를 사용하여 특정 회사의 공시문서를 다운로드하는 함수

    Args:
        corp_name (str): 회사명 (예: '삼성전자')
        start_date (str): 시작일 (YYYYMMDD 형식, 기본값: None)
        period (int): 검색 기간(일) (기본값: 180일)

    Returns:
        str: 작업 결과 메시지

    Example:
    get_dart_documents('삼성전자', None, 120)

    """
    try:
        # .env 파일에서 API 키 로드
        load_dotenv('.env')
        api_key = os.getenv('DART_API_KEY')

        # 고유번호 찾기
        corp_code = get_corp_code(api_key, corp_name)
        if not corp_code:
            return f"{corp_name}의 고유번호를 찾을 수 없습니다."

        # 날짜 설정
        end_date = datetime.now().strftime('%Y%m%d')
        if not start_date:
            start_date = (datetime.now() - timedelta(days=period)).strftime('%Y%m%d')

        # 공시유형 설정 (A: 정기공시)
        doc_types = ['A']

        # 공시문서 검색
        disclosures = search_disclosures(api_key, corp_code, start_date, end_date, doc_types)
        if not disclosures or 'list' not in disclosures:
            return "공시문서를 찾을 수 없습니다."

        # 저장 폴더 설정
        base_folder_name = f"documents_{corp_name}"
        folder_name = base_folder_name
        counter = 1

        # 폴더가 존재하면 번호를 붙여서 새 폴더명 생성
        while os.path.exists(folder_name):
            folder_name = f"{base_folder_name}_{counter}"
            counter += 1

        os.makedirs(folder_name, exist_ok=True)

        # 검색된 공시문서 중 재무 관련 문서만 다운로드
        download_count = 0

        for doc in disclosures['list']:
            rcept_no = doc['rcept_no']
            report_nm = doc['report_nm']

            # 재무 관련 보고서만 다운로드
            if is_financial_report(report_nm):
                # 임시 ZIP 파일 경로
                temp_zip_path = f"{folder_name}/temp_{rcept_no}.zip"

                # 문서 다운로드
                success = download_document(api_key, rcept_no, temp_zip_path)

                if success:
                    # ZIP 파일 압축 풀기 - 별도 폴더 생성 없이 바로 지정된 폴더에 압축 해제
                    with zipfile.ZipFile(temp_zip_path, 'r') as zip_ref:
                        zip_ref.extractall(folder_name)

                    # 임시 ZIP 파일 삭제
                    os.remove(temp_zip_path)

                    download_count += 1

        if download_count > 0:
            return (True, folder_name)
        else:
            return (False, "재무 관련 공시문서가 없습니다.")

    except Exception as e:
        return f"에러가 발생했습니다. {e}"

def get_corp_code(api_key, corp_name):
    """
    회사명으로 고유번호를 찾는 함수

    Args:
        api_key (str): DART API 키
        corp_name (str): 회사명
    Returns:
        str: 고유번호
    """
    url = 'https://opendart.fss.or.kr/api/corpCode.xml'
    params = {
        'crtfc_key': api_key
    }

    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            # XML 응답을 파싱하여 회사명에 해당하는 고유번호 찾기
            # 실제 구현에서는 XML 파싱 라이브러리 사용 필요
            # 예시 코드에서는 임시로 하드코딩된 값 사용
            if corp_name == "삼성전자":
                return "00126380"  # 삼성전자 고유번호
            else:
                return None
    except Exception as e:
        return None

def search_disclosures(api_key, corp_code, start_date=None, end_date=None, doc_types=None):
    """
    공시문서를 검색하는 함수

    Args:
        api_key (str): DART API 키
        corp_code (str): 고유번호
        start_date (str): 시작일 (YYYYMMDD)
        end_date (str): 종료일 (YYYYMMDD)
        doc_types (list): 검색할 공시유형 목록 (기본값: 정기공시)
    Returns:
        list: 검색된 공시문서 리스트
    """
    if not doc_types:
        doc_types = ['A']  # 기본값: 정기공시(A)

    url = 'https://opendart.fss.or.kr/api/list.json'
    params = {
        'crtfc_key': api_key,
        'corp_code': corp_code,
        'bgn_de': start_date,
        'end_de': end_date,
        'page_count': '100'  # 최대 100개까지 검색
    }

    # 공시유형 필터링
    if len(doc_types) == 1:
        params['pblntf_ty'] = doc_types[0]

    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            return None
    except Exception as e:
        return None

def is_financial_report(report_nm):
    """
    보고서가 재무 관련 보고서인지 확인하는 함수

    Args:
        report_nm (str): 보고서명
    Returns:
        bool: 재무 관련 보고서이면 True, 아니면 False
    """
    financial_keywords = [
        '사업보고서', '분기보고서', '반기보고서', '감사보고서',
        '영업(잠정)실적', '매출액', '영업이익', '당기순이익',
        '재무제표', '정기주주총회', '실적발표', '결산실적',
        '전망', '배당', '유상증자', '타법인주식', '투자판단'
    ]

    return any(keyword in report_nm for keyword in financial_keywords)

def download_document(api_key, rcept_no, save_path):
    """
    DART API를 사용하여 공시문서를 다운로드하는 함수

    Args:
        api_key (str): DART API 키
        rcept_no (str): 접수번호
        save_path (str): 저장할 파일 경로
    Returns:
        bool: 다운로드 성공 여부
    """
    url = 'https://opendart.fss.or.kr/api/document.xml'
    params = {
        'crtfc_key': api_key,
        'rcept_no': rcept_no
    }

    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            with open(save_path, 'wb') as f:
                f.write(response.content)
            return True
        else:
            return False
    except Exception as e:
        return False


In [ ]:
corp_name = '삼성전자'

result, dart_dir = get_dart_documents(corp_name, 20240501, 120)
result, dart_dir

(True, 'documents_삼성전자_5')

공시 문서의 형식은 xml이므로, 적절한 방법을 통해 불러와야 합니다.
본 실습에서는 하드코딩된 방법을 사용하지만, 추후 더 좋은 툴이 있다면 해당 툴로 변경하는 것이 높은 성능에 도움이 됩니다.

In [ ]:
from glob import glob
xml_list = glob(f'./{dart_dir}/*.xml')

In [ ]:
# Gemini 2.5 Pro에게 XML 일부를 입력하하고 작성함
import re
import os
from bs4 import BeautifulSoup

def parse_xml(file_path: str) -> str:
    """
    DART 공시 XML 파일을 파싱하여 RAG에 적합한 형식의 텍스트로 변환합니다.

    Args:
        file_path: 파싱할 XML 파일의 경로.

    Returns:
        추출 및 정제된 텍스트 전체를 담은 단일 문자열.
        파일 처리 중 오류 발생 시 빈 문자열을 반환할 수 있습니다.
    """
    extracted_data = []
    current_section_title = "문서 서두" # 기본 섹션 제목

    # --- 1. 파일 존재 확인 및 읽기 ---
    if not os.path.exists(file_path):
        print(f"오류: 파일을 찾을 수 없습니다 - {file_path}")
        return ""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            xml_content = f.read()
    except Exception as e:
        print(f"오류: 파일 읽기 실패 {file_path} - {e}")
        return ""

    # --- 2. XML 파싱 ---
    try:
        # lxml 파서가 설치되어 있다면 속도와 안정성 면에서 더 좋습니다.
        try:
            soup = BeautifulSoup(xml_content, 'lxml-xml')
        except ImportError:
            print("lxml 파서가 없어 내장 'xml' 파서를 사용합니다. 'pip install lxml'로 설치할 수 있습니다.")
            soup = BeautifulSoup(xml_content, 'xml')
    except Exception as e:
        print(f"오류: XML 파싱 실패 {file_path} - {e}")
        return ""

    # --- 3. BODY 태그 찾기 (없으면 전체 문서 처리 시도) ---
    body = soup.find('BODY')
    if not body:
        print(f"경고: {file_path} 파일에서 <BODY> 태그를 찾을 수 없습니다. 문서 전체를 처리합니다.")
        process_root = soup # BODY 없으면 문서 전체를 기준으로 처리
    else:
        process_root = body # BODY 태그 내부를 기준으로 처리

    # --- 4. 내용 순회 및 추출 ---
    try:
        # process_root의 모든 자손 태그를 순회
        for element in process_root.descendants:
            # 유효한 태그 이름이 없는 요소는 건너뛰기 (예: NavigableString)
            if not hasattr(element, 'name') or element.name is None:
                continue

            # --- 섹션 제목 처리 ---
            # 실제 DART 문서의 제목 태그 확인 필요 (예: 'TITLE', 'H2', 'H3' 등)
            # 목차 항목 제외 (ATOC='N' 또는 다른 구분자 확인)
            if element.name == 'TITLE' and element.get('ATOC') != 'N':
                title_text = element.get_text(strip=True)
                # 제목이 비어있지 않고, 이전 제목과 다를 경우 업데이트
                if title_text and title_text != current_section_title:
                     current_section_title = title_text
                     # 마크다운 헤더 형식으로 추가
                     extracted_data.append(f"\n## {current_section_title}\n")

            # --- 문단(P) 처리 ---
            elif element.name == 'P':
                text = element.get_text(strip=True)
                # 문단 내용이 있고, 단순히 숫자만 있는 경우가 아닐 때 추가 (서식용 숫자 제외)
                if text and not text.isdigit():
                    extracted_data.append(text)

            # --- 테이블(TABLE) 처리 ---
            elif element.name == 'TABLE':
                caption_tag = element.find('CAPTION') # 테이블 설명(캡션) 찾기
                caption = caption_tag.get_text(strip=True) if caption_tag else "표" # 캡션 없으면 기본값 '표'

                rows_data = []
                # 테이블의 행(TR 또는 ROW) 찾기 - recursive=False 로 바로 아래 자식만 찾기 시도 가능
                for row in element.find_all(['TR', 'ROW']): # 실제 행 태그 확인 필요
                    # 행 내부의 셀(TD, TH, CELL, TU) 찾기
                    cells = [cell.get_text(strip=True) for cell in row.find_all(['TD', 'TH', 'CELL', 'TU'])] # 실제 셀 태그 확인 필요
                    # 빈 셀 제거
                    cells = [cell for cell in cells if cell]
                    # 셀 내용이 있을 경우에만 행 추가
                    if cells:
                        rows_data.append(" | ".join(cells)) # 파이프(|)로 셀 내용 구분

                # 추출된 행 데이터가 있을 경우에만 테이블 텍스트 생성 및 추가
                if rows_data:
                    table_text = f"\n[{caption}]\n" + "\n".join(rows_data) + "\n"
                    extracted_data.append(table_text)

            # --- 기타 필요한 태그 처리 ---
            # 예: 리스트(UL, OL, LI), 특정 강조(SPAN with attribute) 등 필요시 추가

    except Exception as e:
        print(f"오류: 내용 처리 중 예외 발생 {file_path} - {e}")
        # 부분적으로 추출된 데이터라도 반환할지 결정 (현재는 계속 진행)

    # --- 5. 추출된 텍스트 결합 및 최종 정리 ---
    final_text = "\n".join(extracted_data)

    # 연속된 빈 줄(3개 이상)을 2개로 줄이기
    final_text = re.sub(r'\n{3,}', '\n\n', final_text).strip()

    # 선택적: 테이블 구분선처럼 보이는 라인 제거 (필요시 주석 해제 및 패턴 조정)
    # final_text = "\n".join([line for line in final_text.split('\n') if not re.match(r'^[\s|\-_=]+$', line)])

    return final_text


In [ ]:
from langchain_community.document_loaders import UnstructuredXMLLoader
from langchain.schema import Document
docs = []
for xml in xml_list:
   doc = Document(page_content = parse_xml(xml), metadata={'type':'DART', 'source':xml})
   docs.append(doc)
   print(len(doc.page_content))

29033
39526
26794
44095
91141
104378


In [ ]:
print(docs[0].page_content[20000:21000])

인보우로보틱스㈜ 지분의 전부 또는 일부를 최대주주 등에게 매도할 것을 청구할 권리를 보유하고 있습니다. 2024년 1분기말 현재 해당 콜옵션의 공정가치는 안진회계법인이 평가하였습니다.또한, 종속회사인 삼성디스플레이㈜는 Corning Incorporated와 2021년 4월 8일 실행된 주식매매계약에 따라, 보유 중인 Corning 지분증권 일부를 Corning에게 매각할수 있는 풋옵션을 보유하고 있으며,TCL Technology Group Corporation (TCL) 및 TCL China Star Optoelectronics Technology Co. Ltd. (CSOT)와 2021년 4월 1일 실행된 주주간 계약에 따라 CSOT가 상장기한 내 비상장 시, 삼성디스플레이㈜가 보유 중인 CSOT 지분증권 전부 또는 일부를 TCL에게 매각할 수 있는 풋옵션을 보유하고 있습니다.2024년 1분기말 현재 상기 풋옵션들 공정가치는 한영회계법인이 평가하였습니다.

## 6. 주요계약 및 연구개발활동

가. 경영상의 주요 계약 등

[표]
계약 상대방 | 항  목 | 내   용
Google | 계약 유형 | 상호 특허 사용 계약
체결시기 | 2014.01.25
목적 및 내용 | 상호 특허 라이선스 계약 체결을 통한 사업 자유도 확보
기타 주요내용 | 영구 라이선스 계약 (향후 10년간 출원될 특허까지 포함)
GlobalFoundries | 계약 유형 | 공정 기술 라이선스 계약
체결시기 | 2014.02.28
목적 및 내용 | 14nm 공정의 고객기반 확대
Google | 계약 유형 | EMADA
체결시기 및 기간 | 2019.02.27~2024.12.31(연장)
목적 및 내용 | 유럽 32개국(EEA) 대상으로 Play Store, YouTube 등 구글 앱 사용에 대한라이선스 계약
Ericsson | 계약 유형 | 상호 특허 사용 계약
체결시기 | 2021.05.07
목적 및 내용 | 상호 특허 라이선스 계약 체결을 통한 사업 자유도 확보
Qualcomm | 계약 

각각의 토큰 수도 확인해 보겠습니다.

In [ ]:
import google.generativeai as genai

model = genai.GenerativeModel("models/gemini-2.0-flash")

for doc in docs:
    print(f"FILE: {doc.metadata['source']} ({len(doc.page_content)}) \n {model.count_tokens(doc.page_content)}")

FILE: ./documents_삼성전자_5\20240516001421.xml (29033) 
 total_tokens: 16440

FILE: ./documents_삼성전자_5\20240814003284.xml (39526) 
 total_tokens: 22668

FILE: ./documents_삼성전자_5\20241114002642.xml (26794) 
 total_tokens: 15759

FILE: ./documents_삼성전자_5\20250311001085.xml (44095) 
 total_tokens: 27616

FILE: ./documents_삼성전자_5\20250311001085_00760.xml (91141) 
 total_tokens: 58725

FILE: ./documents_삼성전자_5\20250311001085_00761.xml (104378) 
 total_tokens: 70176



해당 문서를 전부 Context로 넣는 것보다는, 청킹을 통해 필요한 부분만 검색할 수 있도록 구성해 봅시다.   
랭체인에서는 제미나이 토큰 수 기반의 청킹을 지원하지는 않기 때문에, 글자 수 기반의 청킹을 수행합니다.

### 청크 사이즈와 Top K는 어떻게 잡으면 좋을까요?

우리가 사용하는 Gemini 계열의 모델은 1M Context Size기 때문에 많은 내용을 처리할 수 있는데요.   
그러나, Context 길이가 상대적으로 짧은 모델들(128k, 200k 또는 그 이하)의 경우는 청크 사이즈를 작게 만드는 것이 더 효과적일 수 있습니다.   



또한, Context에 포함하기 위한 Top K를 설정하는 것도 중요합니다.   
긴 Context 모델은 Top K를 늘려 풍부한 정보를 파악할 수 있습니다.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Context가 작은 모델이라면, 청크를 줄이는 것이 좋습니다.
# chunk_size = 3000
# chunk_overlap = 600
# top_k = 5

text_splitter = RecursiveCharacterTextSplitter(chunk_size=6000, chunk_overlap=600)

chunks = text_splitter.split_documents(docs)
print(len(chunks))

65


각각의 청크를 벡터 DB에 저장합니다.

In [ ]:
from huggingface_hub import login

# 허깅페이스 토큰 로그인: 아래 코드에서 보통 필요하지 않으나, 필요한 경우 READ TOKEN
login(token=os.environ['HF_TOKEN'])

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [ ]:
# 임베딩 모델 준비 (에러 발생시 위 셀에서 토큰으로 로그인)


from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFaceEmbeddings

model_name = 'intfloat/multilingual-e5-small'
emb_model = SentenceTransformer(model_name, device='cpu')

emb_model.save('./embedding')
del emb_model

import gc
gc.collect()

embeddings = HuggingFaceEmbeddings(model_name= './embedding',
                                   model_kwargs={'device':'cuda'})
# GPU가 있는 경우, CUDA(GPU) 설정

In [ ]:
from langchain_chroma import Chroma

Chroma().delete_collection()
vector_store = Chroma.from_documents(chunks,
                                     collection_name = 'DART',
                                     # persist_directory="./chroma_web",
                                     # 파일 공간에 저장
                                     embedding = embeddings)

retriever = vector_store.as_retriever(search_kwargs={"k": 8})
# Small Model은 작게

c:\Users\pc\venv\multicampus\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


### Hybrid RAG

임베딩 기반 검색도 필요하지만, 전문적인 도메인 문서의 경우에는 키워드 기반의 검색도 필요합니다.   
랭체인의 `BM25Retriever`와 `EnsembleRetriever` 를 이용하여 두 검색을 결합해 보겠습니다.   
한국어 데이터의 경우, 랭체인의 기본 인덱싱에서 처리하지 못하기 때문에    
별도의 형태소 분석기를 추가합니다.

In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

from kiwipiepy import Kiwi
# kiwi 형태소 분석기
kiwi = Kiwi()
def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

bm25_retriever = BM25Retriever.from_documents(chunks, preprocess_func = kiwi_tokenize)
bm25_retriever.k = 5

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever], weights=[0.7, 0.3]
)


<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


NameError: name 'llm' is not defined

LLM을 설정합니다. 이번에는 하나의 모델만 사용하겠습니다.

In [ ]:
import os
from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain.chat_models import init_chat_model
from rich import print as rprint

# Gemini API는 분당 10개 요청으로 제한
# 즉, 초당 약 0.167개 요청 (10/60)
rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.167,  # 분당 10개 요청
    check_every_n_seconds=0.1,  # 100ms마다 체크
    max_bucket_size=10,  # 최대 버스트 크기
)

quick_rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.333,  # 분당 20개 요청
    check_every_n_seconds=0.1,  # 100ms마다 체크
    max_bucket_size=20,  # 최대 버스트 크기
)

llm = init_chat_model(
    model_provider="google_genai",
    model="gemini-2.0-flash",

    rate_limiter=rate_limiter,
    temperature=0.8,
    max_tokens = 8192
)
rprint(llm)

ChatGoogleGenerativeAI(
    rate_limiter=<langchain_core.rate_limiters.InMemoryRateLimiter object at 0x000001CAF3071050>,
    model='models/gemini-2.0-flash',
    google_api_key=SecretStr('**********'),
    temperature=0.8,
    max_output_tokens=8192,
    client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object 
at 0x000001CAA5BEBD90>,
    default_metadata=()
)

In [ ]:
# 결과 비교 (Lexical Win)

question = '''연결회사의 '기타포괄손익-공정가치 금융자산'과 '당기손익-공정가치 금융자산'으로 분류된 상장주식의 주가가 1% 변동할 경우, 2024년 3분기말 기준으로 기타포괄손익과 당기손익에 미치는 영향(세전)은 각각 얼마인가?'''
# 정답: 주가 1% 변동 시 기타포괄손익(법인세효과 반영 전)에 미치는 영향은 58,830백만원이고, 당기손익(법인세효과 반영 전)에 미치는 영향은 2,333백만원 입니다.

for rt in [retriever, bm25_retriever, ensemble_retriever]:
    print(type(rt))
    result = rt.invoke(question)
    context = '\n\n'.join([chunk.page_content for chunk in result])

    print(llm.invoke(f'''
다음 검색 결과를 보고 질문에 답하세요.

{context}

{question}''').content)


    print('-----')

<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>
제공된 자료에서 삼성전자주식회사의 별도재무제표를 기준으로 질문에 대한 답변을 드리겠습니다.

*   **기타포괄손익:** 2024년 3분기말에 대한 정보는 없지만, 2024년 말 기준으로 기타포괄손익-공정가치금융자산의 주가가 1% 변동할 경우 미치는 영향에 대한 정보는 제공되지 않았습니다.
*   **당기손익:** 2024년 3분기말에 대한 정보는 없지만, 2024년 말 기준으로 당기손익-공정가치금융자산의 주가가 1% 변동할 경우 미치는 영향에 대한 정보는 제공되지 않았습니다.

따라서, 제공된 정보만으로는 2024년 3분기말 기준으로 기타포괄손익과 당기손익에 미치는 영향을 정확히 알 수 없습니다.
-----
<class 'langchain_community.retrievers.bm25.BM25Retriever'>
제공된 검색 결과에서 2024년 3분기말 (1분기말) 기준으로 다음 정보를 찾을 수 있습니다.

*   **기타포괄손익**: 주가가 1% 변동 시 58,830백만원의 영향
*   **당기손익**: 주가가 1% 변동 시 2,333백만원의 영향

따라서 2024년 1분기말 기준으로 상장주식의 주가가 1% 변동할 경우, 기타포괄손익에 미치는 영향은 58,830백만원, 당기손익에 미치는 영향은 2,333백만원입니다.
-----
<class 'langchain.retrievers.ensemble.EnsembleRetriever'>
다음과 같습니다.

*   **기타포괄손익:** 58,830 백만원
*   **당기손익:** 2,333 백만원
-----


In [ ]:
# 결과 비교 (Semantic Win)

question = '''매출액 비중이 가장 큰 데가 어디예요? 얼마인가요?'''
# 정답: DX 부문, 매출액은 1조 3,435억 7,500만 원

for rt in [retriever, bm25_retriever, ensemble_retriever]:
    print(type(rt))
    result = rt.invoke(question)
    context = '\n\n'.join([chunk.page_content for chunk in result])

    print(llm.invoke(f'''
다음 검색 결과를 보고 질문에 답하세요.

{context}

{question}''').content)


    print('-----')

<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>
제56기 3분기 매출액 기준으로 매출액 비중이 가장 큰 부문은 DX 부문으로, 1조 3,435,750억 원이며, 이는 전체 매출액의 59.7%를 차지합니다.
-----
<class 'langchain_community.retrievers.bm25.BM25Retriever'>
제공된 검색 결과에서는 삼성전자의 매출액 비중에 대한 직접적인 정보를 찾을 수 없습니다. 재무제표에 대한 감사보고서와 사업 부문별 현황 정보가 있지만, 구체적인 매출액 비중은 포함되어 있지 않습니다. 상세한 매출액 정보는 '상세표-4. 연구개발실적(상세)' 또는 다른 상세 재무 보고서를 참조해야 할 것으로 보입니다.
-----
<class 'langchain.retrievers.ensemble.EnsembleRetriever'>
2024년 3분기 매출액 비중이 가장 큰 부문은 DX 부문으로, 1조 3,435억 7,500만 원입니다.
-----


검색 API를 구성합니다.   
Tavily Search도 툴로 만들어 보겠습니다.

In [ ]:
# Tavily Search

from langchain_tavily import TavilySearch
from typing_extensions import Optional
@tool
def web_search(
    query: str,
    time_range: Optional[str] = None,
    topic: str = "general",
    max_results: int = 5
) -> str:
    """웹 검색을 수행하는 도구입니다.

    Args:
        query: 검색할 쿼리
        time_range: 검색 시간 범위 (None, 'day', 'week', 'month', 'year')
        topic: 검색 주제 ('general', 'finance', 'news')
        max_results: 최대 검색 결과 수 (기본값 5)

    Returns:
        검색 결과 문자열
    """
    tavily_search = TavilySearch(
        max_results=max_results,
        topic=topic,
        include_raw_content=True,
        time_range=time_range
    )

    result = tavily_search.invoke(query)
    return result

llm_with_tools = llm.bind_tools([web_search])
result = llm_with_tools.invoke("삼성전자의 최근 실적에 대한 뉴스 찾아줄래?")
rprint(result)


AIMessage(
    content='',
    additional_kwargs={
        'function_call': {
            'name': 'web_search',
            'arguments': '{"query": "\\uc0bc\\uc131\\uc804\\uc790 \\ucd5c\\uadfc \\uc2e4\\uc801", "time_range": 
"month", "topic": "finance"}'
        }
    },
    response_metadata={
        'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
        'finish_reason': 'STOP',
        'model_name': 'gemini-2.0-flash',
        'safety_ratings': []
    },
    id='run-fda12434-3df4-4fbb-a89e-0f79b73d8192-0',
    tool_calls=[
        {
            'name': 'web_search',
            'args': {'query': '삼성전자 최근 실적', 'time_range': 'month', 'topic': 'finance'},
            'id': 'a9702dc9-0a1d-4890-861f-9f7240615011',
            'type': 'tool_call'
        }
    ],
    usage_metadata={
        'input_tokens': 124,
        'output_tokens': 18,
        'total_tokens': 142,
        'input_token_details': {'cache_read': 0}
    }
)

In [ ]:
# Tavily Search

from langchain_tavily import TavilySearch

# tavily api playground 참고
tavily_search = TavilySearch(
    max_results=5,
    topic="general",
    # include_answer=False,
    include_raw_content=True,
    # include_images=False,
    # include_image_descriptions=False,
    # search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)

In [ ]:
result = tavily_search.invoke("Retrieval Augmented Generation Reasoning")
len(result)

In [ ]:
for i in result['results']:
    print(i['title'])

논문과 테크 리포트를 검색하는 학술 검색 API입니다.

In [ ]:
from langchain_community.retrievers import ArxivRetriever

arxiv_search = ArxivRetriever(
    load_max_docs=5,
    load_all_available_meta=True,
    get_full_documents=True,
    doc_content_chars_max= 100000
    # 10만 글자까지만 수집

)

In [ ]:
docs = arxiv_search.invoke("Retrieval Augmented Generation Reasoning")
# docs

In [ ]:
for doc in docs:
    print(f"Published: {doc.metadata['Published']}")
    print(f"Title: {doc.metadata['Title']}")
    print(f"Authors: {doc.metadata['Authors']}")
    print(f"Summary: {doc.metadata['Summary']}")
    print(f"Length: {len(doc.page_content)}")
    print("-" * 50)



각각의 검색 API를 아래와 같이 정리해 놓겠습니다.

In [ ]:
tool_list = {
    'tavily': tavily_search,
    'arxiv': arxiv_search,
}

전체 과정은 다음과 같이 이루어집니다.

1) 연구 토픽을 입력하면, LLM이 추가 정보를 질문합니다.    
유저는 그대로 진행하거나, 피드백을 전달합니다.

2) 연구 토픽에 대해, LLM이 간단한 검색을 수행하고 이를 바탕으로 연구 개요를 작성합니다.   

3) 개요에 포함된 각 세션에 대해, LLM이 검색 쿼리를 생성하여 각 검색엔진을 통해 검색합니다.   

4) 검색된 결과를 바탕으로 섹션별 내용을 작성합니다.    
(검색 결과에 대한 레퍼런스 표시를 포함합니다.)

5) 섹션별 드래프트를 개선하기 위해, 파생 질문을 추가로 생성하여 더 검색하거나 작성을 종료합니다.


6) 섹션별 내용을 취합하고, 최종 수정을 거친 뒤 리포트를 완성합니다.



In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from langgraph.types import Command, interrupt
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from pydantic import BaseModel, Field
from typing_extensions import Annotated, Literal
import operator
from langgraph.constants import Send

## 작업에 사용할 클래스 만들기   
Structured Output을 위한 클래스를 먼저 구성합니다.

In [ ]:
class Section(BaseModel):
    name: str = Field(description="섹션의 이름")
    description: str = Field(description="해당 섹션에서 다룰 주요 주제에 대한 간략한 개요")
    content: str = Field(description="섹션의 내용 (처음에는 비워 둡니다)")

    @property
    def as_str(self) -> str:
        """섹션의 정보를 포맷팅된 문자열로 변환합니다."""
        return f"### {self.name}\n{self.description}\n\n내용:\n{self.content}"

class ReportPlan(BaseModel):
    sections: list[Section] = Field(description="A list of sections for the report.")
    followup_question: str = Field(description="사용자에게 추가로 질문할 내용 (없으면 '')")

    @property
    def as_str(self) -> str:
        """섹션들을 포맷팅된 문자열로 변환합니다."""
        sections_str = []
        for section in self.sections:
            sections_str.append(f"### {section.name}\n{section.description}")
        return "\n\n".join(sections_str)

## 서브모듈: 토픽에 대한 리서치 모듈 만들기

섹션의 개요가 주어지면, 해당 내용을 검색하여 섹션을 작성하는 과정을 구현해 보겠습니다.

In [ ]:
class ResearchState(TypedDict):
    topic: str
    section: Section
    queries: list[str]
    draft: str
    resources: list
    num_revision: int
    finished: bool


def generate_search_query(state: ResearchState, config: RunnableConfig):
    prompt = ChatPromptTemplate([
('system', f'''
주어진 섹션 정보에 대한 사전 조사를 위해, 효과적인 검색 쿼리를 생성해야 합니다.
해당 주제를 포괄적으로 다룰 수 있는 검색 쿼리들을 생성하세요.

검색 쿼리는 다음과 같은 원칙을 따라야 합니다:
1. 핵심 키워드를 포함해야 합니다
2. 너무 일반적이지 않아야 합니다
3. 학술적이고 전문적인 용어를 사용해야 합니다
4. 최신 연구 동향을 반영해야 합니다
5. 따옴표를 포함하지 않아야 합니다.

적절한 검색 쿼리를 한 줄에 하나씩 작성하세요.
쿼리만 출력하고, {config['configurable']['num_search_queries']} 개의 쿼리를 출력하세요.

'''),
('user', '''
섹션 정보:
{section}

''')])
    section = state['section']

    writer_llm = init_chat_model(
        model_provider = config['configurable']['writer_provider'],
        model= config['configurable']['writer_model'],
        rate_limiter=rate_limiter,
        temperature=0.8,
        max_tokens = 8192
    )


    chain = prompt | writer_llm | StrOutputParser() | (lambda x: x.split('\n'))

    queries = chain.invoke(section.as_str)

    return {'queries':queries}

def search_and_filter(state: ResearchState, config: RunnableConfig):
    '''각각의 검색어에 대해 검색 결과를 수행하고 필터링합니다.
    Tavily Search는 정확성이 높기 때문에, 해당 부분을 생략해도 됩니다..
    빠른 LLM을 사용하겠습니다.
    '''
    quick_llm = init_chat_model(
        model_provider = config['configurable']['quick_provider'],
        model= config['configurable']['quick_model'],
        rate_limiter=quick_rate_limiter,
        temperature=0.8,
    )

    search_tool = tool_list[config['configurable']['search_api'][0]]
    # 적절한 검색 툴 선택 (여기서는 Tavily로 고정)

    queries = state['queries']
    section = state['section']

    relevant_docs=[]


    filter_prompt=ChatPromptTemplate([
        ('system', f'''다음 검색 결과가 주어진 주제와 관련이 있는지 O/X로 판단하세요.
O/X만 출력하세요.
---
주제: {section.as_str}'''),

('user', '''
검색 결과: {doc}''')])
    chain = filter_prompt | quick_llm | StrOutputParser()

    context = search_tool.batch(queries)
    # 모든 검색 쿼리를 한번에 실행

    def preprocess(text):
        import re
        # 탭과 개행문자를 공백으로 변환
        text = text.replace('\t', ' ').replace('\n', ' ').replace('\xa0', ' ')
        # 템플릿 오류 방지
        text = text.replace('{', '(').replace('}', ')')

        # 연속된 공백을 하나로 치환
        text = re.sub(r'\s+', ' ', text).strip()
        return text


    for docs in context:
        try:
            for doc in docs['results']:
                doc_str = f"### {doc['title']} \n URL: {doc['url']} \n {doc['raw_content']}" if doc.get('raw_content') else f"### {doc['title']} \n {doc['content']}"
                # 하나로 만든 뒤 전처리
                doc_str = preprocess(doc_str)
                relevance = chain.invoke(doc_str)
                if relevance=='O':
                    relevant_docs.append(doc_str)
        except: # 검색 오류시
            continue
    print(f'# Filtered Docs: {len(relevant_docs)}')
    return {'resources':relevant_docs}

def write_section(state: ResearchState, config: RunnableConfig):
    section = state['section'].as_str
    topic = state['topic']
    resources = state['resources']
    draft = state.get('draft', '')

    writer_prompt =ChatPromptTemplate([
        ('system', '''
연구 리포트의 주제와 세부 섹션명이 주어집니다.
아래의 정보를 활용하여, 연구 리포트의 한 섹션을 작성하세요.
다음은 작성 가이드라인입니다.

[작성 가이드라인]
간단하고 명확한 언어를 사용하세요.
섹션명은 마크다운 ## 으로 작성하며, 세부 목차는 만들지 말고 문단으로만 분리하세요.
문장을 너무 길게 쓰지 말고, 이해하기 쉽게 작성하세요.
'이다.' 가 아닌 '입니다.', '합니다.' 등의 스타일로 작성하세요.
또한, 아래에 주어지는 정보의 내용을 최대한 활용하여 작성하세요.


[인용 가이드라인]
인용 표시는 [1], [2]와 같이 작성하고, 섹션 마지막에 레퍼런스를 작성하세요.
레퍼런스 형식은 MLA 표기를 따르고, 마지막에 URL도 표시하세요.
예시 표시 형식은 다음과 같습니다.

**References**
[1] Unite.AI. "DeepMind의 Michelangelo 벤치마크: Long-Context LLM의 한계를 드러내다." *Unite.AI*, [https://www.unite.ai/ko/
<br><br>
[2] ...


[노트]
인용 표시를 정확하게 했는지 확인하고, 주장이 기술되는 경우 가급적 소스에 근거하도록 작성하세요.
마크다운 형식을 고려하여, 문단 분리나 레퍼런스 사이의 줄바꿈을 명확하게 하세요.

[기존 드래프트]

기존 드래프트가 주어지는 경우, 여기에 이어서 작성하세요.
'''),

('user',f'''
주제: {topic}

세부 섹션명: {section}

검색 결과 Context:
{resources}

기존 Draft:
{draft}
''')
])
    writer_llm = init_chat_model(
        model_provider = config['configurable']['writer_provider'],
        model= config['configurable']['writer_model'],
        rate_limiter=rate_limiter,
        temperature=0.8,
        max_tokens = 8192
    )
    chain = writer_prompt | writer_llm | StrOutputParser()
    draft = chain.invoke({})
    return {'draft':draft}

class Feedback(BaseModel):
    grade : Literal['Good', 'Bad'] = Field(description='드래프트에 대한 평가')
    queries: Optional[list[str]] = Field(description='검색 쿼리 목록')

def refine_research(state: ResearchState, config: RunnableConfig) -> Command[Literal[END, "search_and_filter"]] :
    '''Draft와 context를 평가하여, 추가 검색이 필요한지 판단합니다.
    Revision 개수를 초과하면 바로 END로 이동합니다.'''

    section = state['section'].as_str
    topic = state['topic']
    resources = state['resources']
    draft = state['draft']
    queries = state['queries']
    num_revision = state['num_revision']

    prompt = ChatPromptTemplate([
        ('system', f'''
연구 리포트의 주제와 세부 섹션명이 주어집니다.

현재 작성된 섹션의 드래프트를 평가하세요.

이 글의 내용을 명확하고 유익하게 작성하기 위해,
검색된 결과 이외의 새로운 내용을 더 조사해야 하는지 판단하세요.

이후, 추가 검색을 위해 필요한 검색어 쿼리를 작성하세요.
{config['configurable']['num_search_queries']} 개 이하의 쿼리를 출력하세요.

해당 글은 충분히 완성도가 높아 추가 조사가 필요하지 않을 수도 있습니다.
그런 경우에는 'Good'을 출력하고, 추가 쿼리를 비워두세요.
'''),

('user',f'''
주제: {topic}

세부 섹션명: {section}


드래프트: {draft}''')
])
    # revision 개수 넘어가면 바로 END
    if num_revision >= config['configurable']['max_search_depth']:
         return Command(goto = END,
                       update = {'finished':True})


    evaluator_llm = init_chat_model(
        model_provider = config['configurable']['evaluator_provider'],
        model= config['configurable']['evaluator_model'],
        rate_limiter=rate_limiter,
        temperature=0.8,
    )


    chain = prompt | evaluator_llm.with_structured_output(Feedback)

    feedback = chain.invoke({})

    if feedback.grade =='Good':
        return Command(goto = END,
                       update = {'finished':True})
    else:
        return Command(goto = search_and_filter,
                       update= {'queries': feedback.queries,
                                'num_revision': num_revision+1})

Research Agent를 구성하는 Small Graph를 만듭니다.

In [ ]:
builder = StateGraph(ResearchState)
builder.add_node(generate_search_query)
builder.add_node(search_and_filter)
builder.add_node(write_section)
builder.add_node(refine_research)


builder.add_edge(START, 'generate_search_query')
builder.add_edge('generate_search_query', 'search_and_filter')
builder.add_edge('search_and_filter', 'write_section')
builder.add_edge('write_section', 'refine_research')

memory = MemorySaver()



researcher_graph = builder.compile(checkpointer=memory)

In [ ]:
researcher_graph

In [ ]:
t = Section(name='Case Study: LLama 4의 Long Context',
            description='Llama 4 모델의 10M Context의 비결에 대해 설명합니다.',
            content='')

In [ ]:
test_research_state={
    'topic': '1M Context Windows 모델',
    'section': t,
    'num_revision':0
}

# Thread
thread = {'configurable':default, 'thread_id':'1'}

history = []
for event in researcher_graph.stream(test_research_state, thread, stream_mode="updates"):
    for status in event:
        print(f'# {status}')
        for key in event[status]:
            value = str(event[status][key])
            if len(value)>300:
                print(f'- {key}: {value[:300]}')
            else:
                print(f'- {key}: {value}')
        print('---------')
    history.append(event)

In [ ]:
load_dotenv('.env', override=True)
# override: 이미 설정된 환경 변수 덮어씌우기

작성한 섹션 드래프트를 확인해 보겠습니다.

In [ ]:
history[-2]

In [ ]:
from IPython.display import display
from IPython.display import Markdown
import textwrap

result = history[-2]['write_section']['draft']

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

to_markdown(result)


섹션별 Writer를 구성했습니다.   
이제 해당 그래프를 서브모듈로 하는 에이전트 구조를 구성합니다.

In [ ]:
class State(TypedDict):
    topic: str # 리포트 주제
    plan: ReportPlan # 리포트 개요
    result: str # 최종 결과물 (하나로 합쳐진)
    human_feedback:str # 개요에 대한 인간 피드백
    finished_drafts: Annotated[list[str], operator.add]
    # (아마도) 병렬 처리로 생성될 draft들을 순서대로 합침


요청을 받은 뒤, 초기 설정과 함께 부가 질문을 수행합니다.

In [ ]:
def initiate_report(state: State , config: RunnableConfig):

    prompt = ChatPromptTemplate([
        ('system','''
당신은 주어진 주제에 대한 연구 보고서의 초기 방향 설정을 위한 개요를 구성합니다.
최대한 최신의 지식과 인사이트를 활용하여야 하며, 사용자를 위한 맞춤형 보고서가 되어야 합니다.

주어진 주제에 대한 섹션별 개요를 작성하세요.
단, 마지막 섹션인 결론은 제외하고 작성하세요.

각 섹션은 불필요한 요소를 포함하지 말아야 하며, 명확하게 구분되어야 합니다.
섹션 간의 겹치는 내용을 최대한 줄이고, 각각의 역할이 분명하도록 구성하세요.

또한, 최종 결과 보고서에 사용자의 선호를 최대한 반영하기 위해,
사용자에게 추가로 질문할 내용도 작성하세요.
예를 들어, 세부 분야, 적용하고자 하는 환경, 원하는 정리 형식 등을 질문할 수 있습니다.

다음은 예시입니다.

---
질문: LLM 파인 튜닝 방법인 LoRA의 최근 발전된 모델들에 대해 조사해줘


답변:
개요: (보고서의 개요)
추가 질문: LoRA 기반 LLM 파인튜닝 관련 최근 발전된 모델들에 대해 조사해드릴게요.
아래 항목들 중 가능한 정보를 알려주시면 더 정확한 조사를 도와드릴 수 있어요:
용도 (예: 챗봇, 코드 생성, 번역, 의료 등)
적용 환경 (예: 연구용, 기업 서비스용, 모바일 디바이스 등)
원하시는 정리 형식 (예: 표, 요약 보고서, 논문 중심 정리 등)
가능한 범위를 알려주시면 곧바로 조사 시작할게요!'''),

('human', '''사용자의 주제(혹은 요청): {topic}

---

관련 최신 검색 결과:
{context}
 ''')
    ])

    topic = state['topic']

    search_tool =  tool_list[config['configurable']['search_api'][0]]
    # 적절한 검색 툴 선택 (여기서는 Tavily로 고정)


    context = search_tool.invoke(topic)
    # 초기 검색을 수행하여 개요 작성

    planner_llm = init_chat_model(
        model_provider = config['configurable']['planner_provider'],
        model= config['configurable']['planner_model'],
        rate_limiter=rate_limiter,
        temperature=0.8,
    ).with_structured_output(ReportPlan)

    chain = prompt | planner_llm

    result = chain.invoke({'topic': topic, 'context' : context})

    print('# Planner: ')

    print(f'''
# 보고서 작성 개요:
{result.as_str}

# 추가 질문:
{result.followup_question}''')
    return {'plan':result}


In [ ]:
def human_review(state: State , config: RunnableConfig):

    human_review = interrupt(
        {
            "question": "피드백을 전달해 주세요, 이대로 진행하고 싶으시면, continue 또는 go만 입력하세요.",
        }
    )
    # Human Feedback을 받아 전달
    review_action = human_review.get("human_feedback")
    return {'human_feedback': review_action}

In [ ]:
def refine_outline(state: State , config: RunnableConfig):
    current_plan = state['plan']
    human_feedback = state['human_feedback']

    if human_feedback.lower()=='go' or human_feedback.lower()=='continue':
        return {'plan': current_plan}

    refine_prompt = PromptTemplate(template='''
보고서의 개요가 주어집니다.
추가 요청사항을 반영하여, 수정된 개요를 작성하세요:

기존 개요:
{current_plan}

---

피드백:
{feedback}

    ''')

    planner_llm = init_chat_model(
        model_provider = config['configurable']['planner_provider'],
        model= config['configurable']['planner_model'],
        rate_limiter=rate_limiter,
        temperature=0.8,
    ).with_structured_output(ReportPlan)

    # 수정된 계획 생성
    refine_chain = refine_prompt | planner_llm

    refined_plan = refine_chain.invoke({
        'current_plan': current_plan.as_str,
        'feedback': human_feedback
    })

    print('# 수정된 보고서 계획:')
    print(f'''
    {refined_plan.as_str}

    추가 질문:
    {refined_plan.followup_question}
    ''')

    return {'plan': refined_plan}

In [ ]:
builder = StateGraph(State)
builder.add_node(initiate_report)
builder.add_node(human_review)
builder.add_node(refine_outline)

builder.add_edge(START, 'initiate_report')
builder.add_edge('initiate_report','human_review')
builder.add_edge('human_review', 'refine_outline')
builder.add_edge('refine_outline', END)
graph = builder.compile(checkpointer= MemorySaver())

In [ ]:
graph

In [ ]:
test_state = {'topic':'Long-Context LLM의 시대'}
thread = {'configurable':default, 'thread_id':'0'}

graph.invoke(test_state, config = thread)

In [ ]:
for event in graph.stream(
        Command(resume={"human_feedback": """오 나 맘바가 궁금해. Mamba에 대한 얘기만 하는 방향으로 수정해줘."""}),
    thread,
    stream_mode="updates", subgraphs=True
):
    history.append(event)
    for status in event:
        print(f'# {str(status)[:300]}')

        try:
            for key in event[status]:
                value = str(event[status][key])
                if len(value)>300:
                    print(f'- {key}: {value[:300]}')
                else:
                    print(f'- {key}: {value}')
            print('---------')
        except:
            continue

In [ ]:
def research(state:ResearchState, config: RunnableConfig):
    result = researcher_graph.invoke({
    'topic': state['topic'],
    'section': state['section'],
    'num_revision':0
    })
    draft = result['draft']

    return {'finished_drafts':[draft]}


def start_survey(state:State, config: RunnableConfig):
    topic = state['topic']
    plan = state['plan']
    # Query 생성, 수집, Reflection, 섹션 작성 모듈을 하나의 에이전트로 구성

    return [Send("research",
            {'topic':topic, "section": s}) for s in plan.sections]


def synthesizer(state:State, config: RunnableConfig):
    return {'result':'\n'.join(state['finished_drafts'])}

def finalizer(state:State, config: RunnableConfig):
    prompt = PromptTemplate(template='''
연구 보고서의 내용이 주어집니다.
전체 흐름을 고려하여, 최종 결론 섹션을 작성하세요.

전체 보고서 내용:
{result}''')
    writer_llm = init_chat_model(
        model_provider = config['configurable']['writer_provider'],
        model= config['configurable']['writer_model'],
        rate_limiter=rate_limiter,
        temperature=0.8,
    )
    chain = prompt | writer_llm | StrOutputParser()
    result = chain.invoke({'result':state['result']})
    return {'result':state['result'] + '\n'+result}


In [ ]:
builder = StateGraph(State)
builder.add_node(initiate_report)
builder.add_node(human_review)
builder.add_node(refine_outline)
builder.add_node(research)
builder.add_node(synthesizer)
builder.add_node(finalizer)

builder.add_edge(START, 'initiate_report')
builder.add_edge('initiate_report','human_review')
builder.add_edge('human_review', 'refine_outline')

builder.add_conditional_edges("refine_outline", start_survey, ["research"])

builder.add_edge('research', 'synthesizer')
builder.add_edge('synthesizer', 'finalizer')
builder.add_edge('finalizer',END)
memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

In [ ]:
graph

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

완성된 그래프를 실행해 보겠습니다.

In [ ]:
test_state = {'topic':'LLM Agent 개발을 위한 프롬프트 엔지니어링의 중요성'}
thread = {'configurable':default, 'thread_id':'1'}

graph.invoke(test_state, config = thread)

In [ ]:
# BottleNeck: Search and Filter
# 긴 컨텍스트 모델은 필터링을 안 하는 방법도 고려할 수 있겠습니다..
'''좋아, Gemini 2.5 Pro는
긴 컨텍스트에서도 성능이 엄청 좋던데, 그 부분을 중요하게 다뤄 주고.
Llama 4의 Long Context에 대해서도 알려줘.        '''

history =[]

for event in graph.stream(
        Command(resume={"human_feedback": """좋아,
지금 흐름 좋은데, 특정 프레임워크에 중점을 두기보다는
전반적인 설계에서 프롬프트를 어떻게 써야 하는지에 대한
에이전트에 특화된 노하우가 들어가면 좋겠어."""}),
    thread,
    stream_mode="updates", subgraphs=True
):
    history.append(event)
    for status in event:
        print(f'# {str(status)[:300]}')

        try:
            for key in event[status]:
                value = str(event[status][key])
                if len(value)>300:
                    print(f'- {key}: {value[:300]}')
                else:
                    print(f'- {key}: {value}')
            print('---------')
        except:
            continue

결과물을 md 파일에 저장해 보겠습니다.

In [ ]:
result = history[-1][1]['finalizer']['result']
result

In [ ]:
with open("example2.md", "w", encoding="utf-8") as f:
    f.write(result)